# **Instalar librerías**

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver


# **Puntoticket**

In [7]:
import sys
import pandas as pd
import datetime

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options = options)

#Guardar url para scraping
driver.get('https://www.puntoticket.com/todos')

#Hora para nombre de archivo
hora = '{:%Y_%b_%d_%H_%M_%S}'.format(datetime.datetime.now())

#Guardar lista de conciertos según el mes
conciertosMes = driver.find_elements(By.CSS_SELECTOR, '#listado-eventos-shuffle > div')

#Crear dataframe vacío con índice
df = pd.DataFrame(columns = ['Nombre', 'Recinto', 'Comuna', 'Tipo_evento', 'Fecha', 'Imagen', 'Url'])

#Ciclo para recorrer eventos por mes
for concierto in conciertosMes:

  conciertoDia = concierto.find_elements(By.CSS_SELECTOR, 'div > div > article')
  i=0

#Ciclo para recorrer evento diarios
  for concierto in conciertoDia:

    nombre = concierto.find_element(By.CSS_SELECTOR, 'a > div.evento--box > div:nth-child(2) > h3').text
    aux =  concierto.find_element(By.CSS_SELECTOR, 'a > div.evento--box > div:nth-child(2) > p.descripcion').text
    recinto = aux.split(' / ')[0].split(' - ')[0]

    try:
      comuna = aux.split(' / ')[0].split(' - ')[-1]
    except:
      comuna = ''

    try:
      tipo_evento = aux.split(' / ')[1]
    except:
      tipo_evento = ''

    fecha = concierto.find_element(By.CSS_SELECTOR, 'a > div.evento--box > div:nth-child(2) > p.fecha').text
    imagen = concierto.find_element(By.CSS_SELECTOR, 'a > div.gallery-inner > img').get_attribute('src')
    url = concierto.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

    df = df.append({ 'Nombre': nombre,
                'Recinto': recinto,
                'Comuna': comuna,
                'Tipo_evento': tipo_evento,
                'Fecha': fecha,
                'Imagen': imagen,
                'Url': url
              }, ignore_index=True )
    i = i + 1

#Convertir dataframe en archivo CSV
df.to_csv(f'Punto_Ticket-{hora}.csv')